# Análise dos imoveis Balneário Camboriú
## Michel Souza Santana
### Data Science Student
> Data: 16/01/2022

In [1063]:
import pandas as pd
import numpy as np
from urllib.request import urlopen
from bs4 import BeautifulSoup as bs

In [1064]:
html = urlopen('https://www.tresas.com.br/todos-os-imoveis')
cf = bs(html.read(), 'html.parser')
dados_imoveis = cf.findAll('a', {'class': 'listing-card-2 max-400 w-inline-block'})
dados_imoveis[:2]

[<a class="listing-card-2 max-400 w-inline-block" href="/imoveis/apartamento-vista-para-o-mar-no-bairro-pioneiros-bc"><div class="listing-2-card-top"><img alt="" class="listing-card-2-photo" height="400" loading="lazy" sizes="(max-width: 479px) 88vw, (max-width: 991px) 380px, (max-width: 1439px) 28vw, 400px" src="https://uploads-ssl.webflow.com/629007f9f06b5eda2376045d/62c08d480babd5d5c5bd6b72_7.jpg" srcset="https://uploads-ssl.webflow.com/629007f9f06b5eda2376045d/62c08d480babd5d5c5bd6b72_7-p-500.jpeg 500w, https://uploads-ssl.webflow.com/629007f9f06b5eda2376045d/62c08d480babd5d5c5bd6b72_7-p-800.jpeg 800w, https://uploads-ssl.webflow.com/629007f9f06b5eda2376045d/62c08d480babd5d5c5bd6b72_7-p-1080.jpeg 1080w, https://uploads-ssl.webflow.com/629007f9f06b5eda2376045d/62c08d480babd5d5c5bd6b72_7.jpg 1280w" width="400"/><div class="listing-2-price"><div class="listing-2-price-text">R$ 7.400.000</div></div></div><div class="listing-content-2"><div><div class="text-medium cod-imovel">#AP0242-BE

In [1065]:
imoveis = []
for dados in dados_imoveis:
    novos_dados = []
    novos_dados.append(dados.find('div', {'class': 'text-large'})) #descrição
    novos_dados.append(dados.find('div', {'class': 'listing-2-price-text'})) #preço
    novos_dados.append(dados.find('div', {'class': 'text-medium margin-top-4'})) #localização
    novos_dados.append(dados.find('div', {'class': 'listing-2-card-features'})) #caracteristicas do imóvel
    novos_dados.append(dados.img['src']) #link imagem
    imoveis.append(novos_dados)

In [1066]:
imoveis_df = pd.DataFrame(imoveis, columns=['descricao_do_imovel', 'preco', 'localizacao', 'caracteristicas_do_imovel', 'link_da_imagem']) 
imoveis_df.head()

,descricao_do_imovel,preco,localizacao,caracteristicas_do_imovel,link_da_imagem
0,[Apartamento com Vista Mar no Bairro Pioneiros...,[R$ 7.400.000],"[Pioneiros, Balneário Camboriú - SC]","[[[3 Suítes]], [[4 Banheiros]], [[]]]",https://uploads-ssl.webflow.com/629007f9f06b5e...
1,"[Apartamento no Diamond Crystal Tower, Centro ...","[R$ 2.989.188,00]","[Rua 1201, 432. Centro - Balneário Camboriú]","[[[4 Suítes]], [[]], [[173,79m²]]]",https://uploads-ssl.webflow.com/629007f9f06b5e...
2,"[Terreno à venda em São Judas, Itajaí ]","[R$ 1.830.000,00]","[Sao Judas, Itajaí - SC]","[[[]], [[]], [[]]]",https://uploads-ssl.webflow.com/629007f9f06b5e...
3,[Apartamento com vista panorâmica e 3 suítes n...,"[R$ 6.341.183,87]","[Praia Brava, Itajaí - SC]","[[[3 Suítes]], [[3 Banheiros]], [[168m² Área P...",https://uploads-ssl.webflow.com/629007f9f06b5e...
4,"[Apartamento com 3 dormitórios sendo 2 suítes,...",[R$ 3.400.000],"[Centro, Balneário Camboriú - SC]","[[[1 Suíte]], [[2 Banheiros]], [[110m² Área Pr...",https://uploads-ssl.webflow.com/629007f9f06b5e...


## Iniciando o processo de limpeza , tratamento e anlise dos dados

In [1067]:
df = imoveis_df
df.shape

(69, 5)

In [1068]:
df.dropna(inplace = True)
df.shape

(69, 5)

In [1069]:
df.size

345

In [1070]:
df.dtypes

descricao_do_imovel          object
preco                        object
localizacao                  object
caracteristicas_do_imovel    object
link_da_imagem               object
dtype: object

In [1071]:
#Eliminando couchetes dos dados
df.head()

,descricao_do_imovel,preco,localizacao,caracteristicas_do_imovel,link_da_imagem
0,[Apartamento com Vista Mar no Bairro Pioneiros...,[R$ 7.400.000],"[Pioneiros, Balneário Camboriú - SC]","[[[3 Suítes]], [[4 Banheiros]], [[]]]",https://uploads-ssl.webflow.com/629007f9f06b5e...
1,"[Apartamento no Diamond Crystal Tower, Centro ...","[R$ 2.989.188,00]","[Rua 1201, 432. Centro - Balneário Camboriú]","[[[4 Suítes]], [[]], [[173,79m²]]]",https://uploads-ssl.webflow.com/629007f9f06b5e...
2,"[Terreno à venda em São Judas, Itajaí ]","[R$ 1.830.000,00]","[Sao Judas, Itajaí - SC]","[[[]], [[]], [[]]]",https://uploads-ssl.webflow.com/629007f9f06b5e...
3,[Apartamento com vista panorâmica e 3 suítes n...,"[R$ 6.341.183,87]","[Praia Brava, Itajaí - SC]","[[[3 Suítes]], [[3 Banheiros]], [[168m² Área P...",https://uploads-ssl.webflow.com/629007f9f06b5e...
4,"[Apartamento com 3 dormitórios sendo 2 suítes,...",[R$ 3.400.000],"[Centro, Balneário Camboriú - SC]","[[[1 Suíte]], [[2 Banheiros]], [[110m² Área Pr...",https://uploads-ssl.webflow.com/629007f9f06b5e...


In [1072]:
df = df.astype(str)

In [1073]:
df['descricao_do_imovel'][0]

'<div class="text-large">Apartamento com Vista Mar no Bairro Pioneiros, BC</div>'

In [1074]:
df['descricao_do_imovel'] = df['descricao_do_imovel'].str.replace('<div class="text-large">','')
df['descricao_do_imovel'] = df['descricao_do_imovel'].str.replace('</div>','')
df['preco'] = df['preco'].str.replace('<div class="listing-2-price-text">','')
df['preco'] = df['preco'].str.replace('</div>','')
df['localizacao'] = df['localizacao'].str.replace('<div class="text-medium margin-top-4">','')
df['localizacao'] = df['localizacao'].str.replace('</div>','')
df['caracteristicas_do_imovel'] = df['caracteristicas_do_imovel'].str.replace('<div class="listing-2-card-features">','')
df['caracteristicas_do_imovel'] = df['caracteristicas_do_imovel'].str.replace('</div>','')
df['caracteristicas_do_imovel'] = df['caracteristicas_do_imovel'].str.replace('<div class="features first-item">','')
df['caracteristicas_do_imovel'] = df['caracteristicas_do_imovel'].str.replace('</div>','')
df['caracteristicas_do_imovel'] = df['caracteristicas_do_imovel'].str.replace('<div class="text-medium">','/')
df['caracteristicas_do_imovel'] = df['caracteristicas_do_imovel'].str.replace('</div>','')
df['caracteristicas_do_imovel'] = df['caracteristicas_do_imovel'].str.replace('<div class="text-medium w-dyn-bind-empty">','/')
df['caracteristicas_do_imovel'] = df['caracteristicas_do_imovel'].str.replace('</div>','')

In [1075]:
df.head()

,descricao_do_imovel,preco,localizacao,caracteristicas_do_imovel,link_da_imagem
0,"Apartamento com Vista Mar no Bairro Pioneiros, BC",R$ 7.400.000,"Pioneiros, Balneário Camboriú - SC",/3 Suítes/4 Banheiros/,https://uploads-ssl.webflow.com/629007f9f06b5e...
1,"Apartamento no Diamond Crystal Tower, Centro d...","R$ 2.989.188,00","Rua 1201, 432. Centro - Balneário Camboriú","/4 Suítes//173,79m²",https://uploads-ssl.webflow.com/629007f9f06b5e...
2,"Terreno à venda em São Judas, Itajaí","R$ 1.830.000,00","Sao Judas, Itajaí - SC",///,https://uploads-ssl.webflow.com/629007f9f06b5e...
3,Apartamento com vista panorâmica e 3 suítes na...,"R$ 6.341.183,87","Praia Brava, Itajaí - SC",/3 Suítes/3 Banheiros/168m² Área Privativa,https://uploads-ssl.webflow.com/629007f9f06b5e...
4,"Apartamento com 3 dormitórios sendo 2 suítes, ...",R$ 3.400.000,"Centro, Balneário Camboriú - SC",/1 Suíte/2 Banheiros/110m² Área Privativa,https://uploads-ssl.webflow.com/629007f9f06b5e...


In [1076]:
df['preco'] = df['preco'].str.replace('[R$ ]','')

/tmp/ipykernel_47736/3507475529.py:1: FutureWarning: The default value of regex will change from True to False in a future version.
  df['preco'] = df['preco'].str.replace('[R$ ]','')


In [1077]:
df.head()

,descricao_do_imovel,preco,localizacao,caracteristicas_do_imovel,link_da_imagem
0,"Apartamento com Vista Mar no Bairro Pioneiros, BC",7.400.000,"Pioneiros, Balneário Camboriú - SC",/3 Suítes/4 Banheiros/,https://uploads-ssl.webflow.com/629007f9f06b5e...
1,"Apartamento no Diamond Crystal Tower, Centro d...","2.989.188,00","Rua 1201, 432. Centro - Balneário Camboriú","/4 Suítes//173,79m²",https://uploads-ssl.webflow.com/629007f9f06b5e...
2,"Terreno à venda em São Judas, Itajaí","1.830.000,00","Sao Judas, Itajaí - SC",///,https://uploads-ssl.webflow.com/629007f9f06b5e...
3,Apartamento com vista panorâmica e 3 suítes na...,"6.341.183,87","Praia Brava, Itajaí - SC",/3 Suítes/3 Banheiros/168m² Área Privativa,https://uploads-ssl.webflow.com/629007f9f06b5e...
4,"Apartamento com 3 dormitórios sendo 2 suítes, ...",3.400.000,"Centro, Balneário Camboriú - SC",/1 Suíte/2 Banheiros/110m² Área Privativa,https://uploads-ssl.webflow.com/629007f9f06b5e...


In [1083]:
df['caracteristicas_do_imovel'] = df['caracteristicas_do_imovel'].str.split("/", expand=False)

In [1084]:
df.head()

,descricao_do_imovel,preco,localizacao,caracteristicas_do_imovel,link_da_imagem
0,"Apartamento com Vista Mar no Bairro Pioneiros, BC",7.400.000,"Pioneiros, Balneário Camboriú - SC",NaN,https://uploads-ssl.webflow.com/629007f9f06b5e...
1,"Apartamento no Diamond Crystal Tower, Centro d...","2.989.188,00","Rua 1201, 432. Centro - Balneário Camboriú",NaN,https://uploads-ssl.webflow.com/629007f9f06b5e...
2,"Terreno à venda em São Judas, Itajaí","1.830.000,00","Sao Judas, Itajaí - SC",NaN,https://uploads-ssl.webflow.com/629007f9f06b5e...
3,Apartamento com vista panorâmica e 3 suítes na...,"6.341.183,87","Praia Brava, Itajaí - SC",NaN,https://uploads-ssl.webflow.com/629007f9f06b5e...
4,"Apartamento com 3 dormitórios sendo 2 suítes, ...",3.400.000,"Centro, Balneário Camboriú - SC",NaN,https://uploads-ssl.webflow.com/629007f9f06b5e...
